In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split # function for splitting data to train and test sets

import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline


ModuleNotFoundError: No module named 'wordcloud'

In [2]:
train_data=pd.read_csv(r"C:\Users\gaurav.singh.rawal\Documents\Gaurav\Data Science\Machine Learning-Predictive Analytics\ML-Projects\GIT - ML - Projects\ML_Projects\NLP - Sentiment Analysis\HackerRank - Mothers Day\train.csv")
test_data=pd.read_csv(r"C:\Users\gaurav.singh.rawal\Documents\Gaurav\Data Science\Machine Learning-Predictive Analytics\ML-Projects\GIT - ML - Projects\ML_Projects\NLP - Sentiment Analysis\HackerRank - Mothers Day\test.csv")

In [3]:
train_data.head()

,id,original_text,lang,retweet_count,original_author,sentiment_class
0,1.245025e+18,Happy #MothersDay to all you amazing mothers o...,en,0,BeenXXPired,0
1,1.245759e+18,Happy Mothers Day Mum - I'm sorry I can't be t...,en,1,FestiveFeeling,0
2,1.246087e+18,Happy mothers day To all This doing a mothers ...,en,0,KrisAllenSak,-1
3,1.244803e+18,Happy mothers day to this beautiful woman...ro...,en,0,Queenuchee,0
4,1.244876e+18,Remembering the 3 most amazing ladies who made...,en,0,brittan17446794,-1


In [4]:
test_data.head()

,id,original_text,lang,retweet_count,original_author
0,1.246628e+18,"3. Yeah, I once cooked potatoes when I was 3 y...",en,0,LToddWood
1,1.245898e+18,"Happy Mother's Day to all the mums, step-mums,...",en,0,iiarushii
2,1.244717e+18,"I love the people from the UK, however, when I...",en,0,andreaanderegg
3,1.245730e+18,Happy 81st Birthday Happy Mother’s Day to my m...,en,1,TheBookTweeters
4,1.244636e+18,Happy Mothers day to all those wonderful mothe...,en,0,andreaanderegg


In [5]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

In [6]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [7]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

In [8]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [9]:
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

In [10]:
def lemmatize_text(text):
    word_list = nltk.word_tokenize(text)
    lemmatizer = WordNetLemmatizer() 
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])

    return lemmatized_output

In [11]:
def remove_stopwords(text, is_lower_case=False):
    stop_words = set(stopwords.words('english')) 
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stop_words]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [12]:
def normalize_corpus(review, html_stripping=True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
        # strip HTML
        if html_stripping:
            review = strip_html_tags(review)
        # remove accented characters
        if accented_char_removal:
            review = remove_accented_chars(review)
        # expand contractions    
        if contraction_expansion:
            review = expand_contractions(review)
        # lowercase the text    
        if text_lower_case:
            review = review.lower()
        # remove extra newlines
        review = re.sub(r'[\r|\n|\r\n]+', ' ',review)
        # lemmatize text
        if text_lemmatization:
            review = lemmatize_text(review)
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            review = special_char_pattern.sub(" \\1 ", review)
            review = remove_special_characters(review, remove_digits=remove_digits)  
        # remove extra whitespace
        review = re.sub(' +', ' ', review)
        # remove stopwords
        if stopword_removal:
            review = remove_stopwords(review, is_lower_case=text_lower_case)
            
        return review

In [13]:
train_data['original_text']=train_data['original_text'].apply(normalize_corpus)

In [14]:
test_data['original_text']=test_data['original_text'].apply(normalize_corpus)

In [15]:
# Create feature vectors
vectorizer = TfidfVectorizer(min_df=5,
                                 max_df = 0.8,
                                 sublinear_tf=True,
                                 use_idf=True)


In [16]:
train_vectors = vectorizer.fit_transform(train_data['original_text'])
test_vectors = vectorizer.transform(test_data['original_text'])

In [17]:
train_vectors_df=pd.DataFrame(train_vectors.toarray(), columns=vectorizer.get_feature_names())
test_vectors_df=pd.DataFrame(test_vectors.toarray(), columns=vectorizer.get_feature_names())

In [18]:
train_df=pd.concat([train_vectors_df,train_data[['retweet_count','sentiment_class','lang']]],axis=1)
test_df=pd.concat([test_vectors_df,test_data[['retweet_count','lang']]],axis=1)

In [19]:
train_df=train_df[train_df['lang']=='en']
test_df=test_df[test_df['lang']=='en']

In [22]:
train_df=train_df.drop('lang',axis=1)
test_df=test_df.drop('lang',axis=1)

In [24]:
y=train_df['sentiment_class']
X=train_df.drop('sentiment_class',axis=1)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [26]:
# Perform classification with MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)


In [27]:
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

          -1       0.20      0.05      0.08       202
           0       0.50      0.92      0.64       447
           1       0.35      0.02      0.04       250

    accuracy                           0.48       899
   macro avg       0.35      0.33      0.26       899
weighted avg       0.39      0.48      0.35       899



In [28]:
print("\nAccuracy score of Multinomial naive bayes algorithm -----> " + str(accuracy_score(y_test, prediction)))


Accuracy score of Multinomial naive bayes algorithm -----> 0.4760845383759733


In [29]:
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
classifier_linear.fit(X_train, y_train)
prediction_linear = classifier_linear.predict(X_test)


In [30]:
print(classification_report(y_test, prediction_linear))

              precision    recall  f1-score   support

          -1       0.21      0.09      0.12       202
           0       0.50      0.83      0.62       447
           1       0.35      0.10      0.15       250

    accuracy                           0.46       899
   macro avg       0.35      0.34      0.30       899
weighted avg       0.39      0.46      0.38       899



In [31]:
print("\nAccuracy score of Multinomial naive bayes algorithm -----> " + str(accuracy_score(y_test, prediction_linear)))


Accuracy score of Multinomial naive bayes algorithm -----> 0.4593993325917686
